<a href="https://colab.research.google.com/github/myprogrammerpersonality/Table2Speech/blob/main/Table2Speech_vDev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Important Notes
1. your volume file must have this format:<br>
  row1: metabolite 1, metabolite 2, metabolite 3, ...,  metabolite m<br>
row2 to n: volume,    volume,       volume, ...,  volume<br> 

2. after uploading your file, remove Template.csv and rename your file to Template

## Installing Required Packages

In [ ]:
!pip install gtts
!mkdir sounds

  Created wheel for gtts-token: filename=gTTS_token-1.1.3-cp36-none-any.whl size=4096 sha256=bc5deaa2d32e754a07139def436ecaae61baee0b91c7bf46416b3ad9ba7300ba
  Stored in directory: /root/.cache/pip/wheels/dd/11/61/33f7e51bf545e910552b2255eead2a7cd8ef54064b46dceb34
Successfully built gtts-token
--2020-10-11 20:27:05--  https://raw.githubusercontent.com/myprogrammerpersonality/Voice_Recognition/master/Template.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 267 [text/plain]
Saving to: ‘Template.csv’

Template.csv        100%[===================>]     267  --.-KB/s    in 0s      

2020-10-11 20:27:05 (8.67 MB/s) - ‘Template.csv’ saved [267/267]



## Import Packages and Define Functions

In [ ]:
import scipy
from scipy.io.wavfile import read as wav_read
import io
from IPython.display import HTML, Audio, clear_output
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import IPython.display as ipd
from IPython.display import Javascript
import matplotlib.pyplot as plt
import time
import pandas as pd
from gtts import gTTS #Import Google Text to Speech

output_html = """<style>
    fieldset {{
    font-family: sans-serif;
    border: 5px solid #1F497D;
    background: #ddd;
    border-radius: 5px;
    padding: 15px;
}}

fieldset legend {{
    background: #1F497D;
    color: #fff;
    padding: 5px 10px ;
    font-size: 32px;
    border-radius: 10px;
    box-shadow: 0 0 0 5px #ddd;
    margin-left: 20px;
}}
</style>

<section style="margin: 15px;">
<fieldset style="min-height:100px;">
<legend><b> {} </b> </legend>
<label> <h1 style="font-size: 80px;float: top;">{} ==> Sample {}</h1><br/> </label>
</fieldset>"""


## Text to Speech

In [ ]:
data = pd.read_csv('Template.csv')
columns = list(range(1, len(data)+1))
metabolite = data.columns
data = pd.DataFrame(columns=columns, data=data.values.T)
data['Metabolite'] = metabolite
data = data[['Metabolite']+columns]
data

,Metabolite,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,PEG,600.0,1800.0,2000.0,1400.0,1400.0,1200.0,1200.0,1800.0,1400.0,400.0,400.0,800.0,2000.0,1800.0,1200.0,1800.0,1800.0,1600.0,1400.0,600.0
1,DTT,400.0,0.0,100.0,300.0,800.0,400.0,400.0,700.0,200.0,300.0,300.0,0.0,800.0,300.0,800.0,600.0,300.0,800.0,500.0,500.0
2,Spermidine,1000.0,400.0,500.0,100.0,200.0,800.0,1000.0,200.0,100.0,1000.0,200.0,800.0,1000.0,700.0,500.0,700.0,700.0,800.0,500.0,900.0
3,T7Pol,1300.0,1000.0,1600.0,1300.0,1000.0,1800.0,200.0,2000.0,2000.0,1300.0,1800.0,2000.0,200.0,2000.0,1600.0,400.0,200.0,1600.0,1800.0,1800.0
4,t-RNA,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
5,AminoAcids,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0
6,TF-plasmid,400.0,50.0,400.0,800.0,400.0,50.0,100.0,600.0,400.0,400.0,600.0,400.0,800.0,400.0,400.0,400.0,100.0,800.0,800.0,50.0
7,pSTAR,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,0.0,0.0,0.0,0.0
8,pTHS,0.0,200.0,0.0,200.0,0.0,200.0,0.0,200.0,0.0,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,pTAC,0.0,0.0,200.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,200.0


In [ ]:
main_dict = {}
for name in data['Metabolite']:
    vols = list(data[data['Metabolite']==name].iloc[:,1:].values[0])
    main_dict[name] = [vols, sorted(range(len(vols)), key=lambda k: vols[k])]

for name in main_dict.keys():
    print(f'reading {name} ...')
    tts = gTTS('Start Aliquoting {}'.format(name)) #Provide the string to convert to speech
    tts.save('sounds/{}.wav'.format(name)) #save the string converted to speech as a .wav file
    for i, vol in enumerate(main_dict[name][0]):
        tts = gTTS('{} in Sample {}'.format(vol, i+1))
        tts.save('sounds/{}_{}.wav'.format(name, i))

## Main Part KeyBoard Control

In [ ]:
# start from 1
index = 0

First = True
total = sum([len(i[0]) for i in main_dict.values()])
each = len(list(main_dict.values())[0][0])

In [ ]:
in_row = index % each
in_col = index // each
print(index, in_col, in_row)
name = list(main_dict.keys())[in_col]

if in_row == 0:
    display(Audio('sounds/{}.wav'.format(name), autoplay=True))
    display(HTML(output_html.format(name, "#", "#")))
    time.sleep(4)
    clear_output(wait=True)
    time.sleep(1)

if index <= total: 
    display(Audio('sounds/{}_{}.wav'.format(name, main_dict[name][1][in_row]), autoplay=True))
    display(HTML(output_html.format(name, main_dict[name][0][main_dict[name][1][in_row]], main_dict[name][1][in_row]+1)))
else:
    print('The End')

# go to next one
index += 1
First = True

6 0 6


In [ ]:
# Go One Back
if First:
    index -= 2
    First = False
else:
    index -= 1